In [1]:
using JuMP, GLPK
using DataFrames, CSV, XLSX

In [2]:
# DATA: FISH 
df_fish = DataFrame(CSV.File("fish_data.csv"))
fish_price = df_fish."2021 Market Price"
fish_health_value = df_fish."Protein Value"

# DATA: FISHING GROUND
df_fishingground_max = DataFrame(CSV.File("fishingground_restriction.csv"))

# DATA: PROCESSING FACILITY
df_processingfacility = DataFrame(CSV.File("processing_facility.csv"))
processing_cap = df_processingfacility."Processing Capacity"
operational_cost = df_processingfacility."Operational Cost"

# DATA: CITIES
df_cities = DataFrame(CSV.File("cities_data.csv"))
cities_budget = df_cities."GDP per Capita"
cities_population = df_cities."Population"

# DATA FOR OBJECTIVE FUNCTION
fishing_cost_3d = [
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 1")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 2")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 3")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 4")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 5")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 6")...), 
    DataFrame(XLSX.readtable("fishing_cost.xlsx", "Fish 7")...)
]
transportation_cost_3d = [
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 1")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 2")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 3")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 4")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 5")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 6")...), 
    DataFrame(XLSX.readtable("transportation_cost.xlsx", "Fish 7")...)
]

7-element Vector{DataFrame}:
 53×100 DataFrame
 Row │ 1        2        3        4        5        6        7        8        ⋯
     │ Any      Any      Any      Any      Any      Any      Any      Any      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 5.27071  8.72569  722.814  817.425  1.82021  318.987  472.962  21.0398  ⋯
   2 │ 477.028  477.762  354.176  336.004  481.554  736.815  13.2156  466.454
   3 │ 9.7982   12.1435  709.861  802.578  14.1971  328.518  458.088  12.6319
   4 │ 13.5411  11.3988  711.857  801.814  16.3915  324.595  457.229  5.94622
   5 │ 652.188  653.301  278.171  160.175  656.819  912.69   185.206  642.286  ⋯
   6 │ 11.2093  11.3621  728.861  823.1    6.33721  313.494  478.599  24.3848
   7 │ 8.63362  14.5472  709.208  804.318  13.6842  331.104  459.939  18.3479
   8 │ 13.1959  20.3829  705.998  803.645  17.8962  336.129  459.431  24.6925
   9 │ 1.73936  8.80826  718.929  813.707  4.21076  322.456  469.267  19.4037  ⋯

In [3]:
# PARAMETERS
F = 7       # number of fish types
G = 16       # number of fishing grounds
I = 53       # number of processing facilities
J = 100     # number of cities

# VARIABLES
m = Model(GLPK.Optimizer)
@variable(m, x[1:F, 1:G, 1:I])
@variable(m, y[1:F, 1:I, 1:J])
@variable(m, z[1:I], Bin)
@variable(m, s[1:J])

100-element Vector{VariableRef}:
 s[1]
 s[2]
 s[3]
 s[4]
 s[5]
 s[6]
 s[7]
 s[8]
 s[9]
 s[10]
 s[11]
 s[12]
 s[13]
 ⋮
 s[89]
 s[90]
 s[91]
 s[92]
 s[93]
 s[94]
 s[95]
 s[96]
 s[97]
 s[98]
 s[99]
 s[100]

In [4]:
# CONSTRAINTS
# Non-negativity constraints
for f in 1:F
    for g in 1:G
        for i in 1:I
            @constraint(m, x[f, g, i] >= 0)
        end
    end
end
for f in 1:F
    for i in 1:I
        for j in 1:J
            @constraint(m, y[f, i, j] >= 0)
        end
    end
end
for j in 1:J
    @constraint(m, s[j] >= 0)
end      

# Fishing Restriction Constraint
for f in 1:F
    for g in 1:G
        @constraint(m, sum(x[f, g, i] for i in 1:I) <= df_fishingground_max[g, f])
    end
end

# Processing Capacity Constraint
for i in 1:I
    @constraint(m, sum(y[f, i, j] for f in 1:F for j in 1:J) <= processing_cap[i])
end

# Protein Consumption Constraint 
for j in 1:J 
    @constraint(m, sum(y[f, i, j]*fish_health_value[f] for f in 1:F for i in 1:I) <= 43800 * cities_population[j])
    # @constraint(m, sum(y[f, i, j]*fish_health_value[f] for f in 1:F for i in 1:I) >= 17520 * cities_population[j])
    @constraint(m, sum(y[f, i, j]*fish_health_value[f] for f in 1:F for i in 1:I) >= 5840 * cities_population[j])
end

# Distribution Capacity Constraint
for f in 1:F
    for i in 1:I
        @constraint(m, sum(y[f, i, j] for j in 1:J) <= sum(x[f, g, i] for g in 1:G))
    end
end

# Budget Constraint
for j in 1:J
    @constraint(m, sum(y[f, i, j] * fish_price[f] for f in 1:F for i in 1:I)  <= cities_population[j] * (0.15) * cities_budget[j] + s[j])
end

# Fish Diversity Constraint
for f in 1:F
    for j in 1:J
        @constraint(m, 0.025 * sum(y[f_sum_index, i, j] for f_sum_index in 1:F for i in 1:I) <= sum(y[f, i, j] for i in 1:I))
        @constraint(m, 0.25 * sum(y[f_sum_index, i, j] for f_sum_index in 1:F for i in 1:I) >= sum(y[f, i, j] for i in 1:I))
    end
end

# # Fishing Ground Diversity Constraint
for g in 1:G
    @constraint(m, 0.025 * sum(x[f, g_sum_index, i] for f in 1:F for g_sum_index in 1:G for i in 1:I) <= sum(x[f,g,i] for f in 1:F for i in 1:I))
    @constraint(m, 0.25 * sum(x[f, g_sum_index, i] for f in 1:F for g_sum_index in 1:G for i in 1:I) >= sum(x[f,g,i] for f in 1:F for i in 1:I))
end 

# Facility Opening Constraint
# Part 1
for i in 1:I
    @constraint(m, sum(x[f,g,i] for f in 1:F for g in 1:G) <= sum(z[i] * df_fishingground_max[g, f] for f in 1:F for g in 1:G) )
end
# Part 2
for i in 1:I
    @constraint(m, sum(y[f,i,j] for f in 1:F for j in 1:J) <= z[i] * processing_cap[i])
end



In [5]:
# OBJECTIVE FUNCTION
@objective(m, Max, (10^5)*sum(fish_health_value[f]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(fishing_cost_3d[f][g, i]*x[f, g, i] for f in 1:F for g in 1:G for i in 1:I) - sum(transportation_cost_3d[f][i,j]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(operational_cost[i]*z[i] for i in 1:I) - sum(s[j] for j in 1:J))

2.635299472929424e7 y[1,1,1] + 2.6352991274306633e7 y[1,1,2] + 2.6352277186031885e7 y[1,1,3] + 2.6352182574782725e7 y[1,1,4] + 2.6352998179786727e7 y[1,1,5] + 2.635268101329086e7 y[1,1,6] + 2.6352527037985224e7 y[1,1,7] + 2.6352978960233998e7 y[1,1,8] + 2.635298528714414e7 y[1,1,9] + 2.635298617850333e7 y[1,1,10] + 2.635233899648835e7 y[1,1,11] + 2.6352994364998396e7 y[1,1,12] + 2.6352970744886536e7 y[1,1,13] + 2.635212371200539e7 y[1,1,14] + 2.6352986041531913e7 y[1,1,15] + 2.6352982945921373e7 y[1,1,16] + 2.6352996887060355e7 y[1,1,17] + 2.6352982328614067e7 y[1,1,18] + 2.6352989535757996e7 y[1,1,19] + 2.63529816518661e7 y[1,1,20] + 2.6352591142931625e7 y[1,1,21] + 2.6352608826688558e7 y[1,1,22] + 2.6352973803989187e7 y[1,1,23] + 2.6352955945750766e7 y[1,1,24] + 2.6352516660609994e7 y[1,1,25] + 2.635297993788931e7 y[1,1,26] + 2.6352943369970996e7 y[1,1,27] + 2.6352986582033068e7 y[1,1,28] + 2.6352974595779434e7 y[1,1,29] + 2.6352983347004127e7 y[1,1,30] + [[...43129 terms omitted...]

In [6]:
optimize!(m)

Error: basis matrix is singular to working precision (cond = 3.29e+016)


In [7]:
solution_summary(m)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 1.02286e+17
  Objective bound    : 1.02286e+17
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.35878e+02


In [8]:
objective_value(m)

1.0228604232602997e17

In [9]:
println(value.(x))

[0.0 0.0 0.0 0.0 0.0 0.0 1.2899702735382862e7 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 4.573464242010518e6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 3.2744919926066156e6 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4.215044964993552e7 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4.2150449649935514e7 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2.840917621176123e7 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4.215044964993552e6 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4.215044964993552e6 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0

In [10]:
# NEW PRINTER
# Print the optimal solution for each decision variable
println("Optimal solution for x:")
for f in 1:F
    for g in 1:G
        for i in 1:I
            if value(x[f, g, i]) != 0
                println("x[$f, $g, $i] = ", value(x[f, g, i]))
            end
        end
    end
end

println("\nOptimal solution for y:")
for f in 1:F
    for i in 1:I
        for j in 1:J
            if value(y[f, i, j]) != 0
                println("y[$f, $i, $j] = ", value(y[f, i, j]))
            end
        end
    end
end

println("\nOptimal solution for z:")
for i in 1:I
    if value(z[i]) != 0
        println("z[$i] = ", value(z[i]))
    end
end

println("\nOptimal solution for s:")
for j in 1:J
    if value(s[j]) != 0
        println("s[$j] = ", value(s[j]))
    end
end


Optimal solution for x:
x[1, 1, 3] = 3.092858670249685e7
x[1, 1, 8] = 7.984940947921789e6
x[1, 1, 12] = 4.7410299670132294e7
x[1, 1, 13] = 1.1860902216718495e7
x[1, 1, 15] = 6.292169998580135e6
x[1, 1, 16] = 5.3170941605245546e7
x[1, 1, 24] = 7.485465570332318e6
x[1, 1, 27] = 2.0425727633855015e7
x[1, 1, 53] = 2.3167078547175676e6
x[1, 2, 40] = 1.0370190825959444e8
x[1, 2, 48] = 1.285800049367464e7
x[1, 5, 45] = 1.12823312875e7
x[1, 6, 32] = 4.849554649985436e7
x[1, 6, 33] = 2.2750627087665997e7
x[1, 6, 34] = 1.0017556070944768e8
x[1, 6, 38] = 1.6128774905969808e6
x[1, 6, 50] = 3.381523136634283e7
x[1, 6, 53] = 4.0151093733901046e6
x[1, 7, 1] = 1.2899702735382862e7
x[1, 7, 4] = 7.2287561020579655e6
x[1, 7, 18] = 4.428875441762432e7
x[1, 7, 19] = 3.903108521386272e7
x[1, 7, 20] = 2.904290590471172e6
x[1, 7, 26] = 1.5774410940600961e7
x[1, 8, 28] = 4.429717987873915e7
x[1, 9, 31] = 3.456266463291195e7
x[1, 10, 2] = 4.215044964993552e7
x[1, 10, 14] = 3.596668944165078e7
x[1, 10, 42] = 5.6

In [13]:
using DataFrames

# Create an empty DataFrame to store the results
results_df = DataFrame(Variable = String[], Value = Float64[])

# Iterate over x variables
for f in 1:F
    for g in 1:G
        for i in 1:I
            if value(x[f, g, i]) != 0
                push!(results_df, ["x[$f, $g, $i]", value(x[f, g, i])])
            end
        end
    end
end

# Iterate over y variables
for f in 1:F
    for i in 1:I
        for j in 1:J
            if value(y[f, i, j]) != 0
                push!(results_df, ["y[$f, $i, $j]", value(y[f, i, j])])
            end
        end
    end
end

# Iterate over z variables
for i in 1:I
    if value(z[i]) != 0
        push!(results_df, ["z[$i]", value(z[i])])
    end
end

# Iterate over s variables
for j in 1:J
    if value(s[j]) != 0
        push!(results_df, ["s[$j]", value(s[j])])
    end
end

# Print or export the DataFrame
println(results_df)
CSV.write("results.csv", results_df)


1087×2 DataFrame
  Row │ Variable       Value          
      │ String         Float64        
──────┼───────────────────────────────
    1 │ x[1, 1, 3]          3.09286e7
    2 │ x[1, 1, 8]          7.98494e6
    3 │ x[1, 1, 12]         4.74103e7
    4 │ x[1, 1, 13]         1.18609e7
    5 │ x[1, 1, 15]         6.29217e6
    6 │ x[1, 1, 16]         5.31709e7
    7 │ x[1, 1, 24]         7.48547e6
    8 │ x[1, 1, 27]         2.04257e7
    9 │ x[1, 1, 53]         2.31671e6
   10 │ x[1, 2, 40]         1.03702e8
   11 │ x[1, 2, 48]         1.2858e7
   12 │ x[1, 5, 45]         1.12823e7
   13 │ x[1, 6, 32]         4.84955e7
   14 │ x[1, 6, 33]         2.27506e7
   15 │ x[1, 6, 34]         1.00176e8
   16 │ x[1, 6, 38]         1.61288e6
   17 │ x[1, 6, 50]         3.38152e7
   18 │ x[1, 6, 53]         4.01511e6
   19 │ x[1, 7, 1]          1.28997e7
   20 │ x[1, 7, 4]          7.22876e6
   21 │ x[1, 7, 18]         4.42888e7
   22 │ x[1, 7, 19]         3.90311e7
   23 │ x[1, 7, 20]         2.9

"results.csv"

In [11]:
# # Print the optimal solution for each decision variable
# println("Optimal solution for x:")
# for f in 1:F
#     for g in 1:G
#         for i in 1:I
#             println("x[$f, $g, $i] = ", value(x[f, g, i]))
#         end
#     end
# end

# println("\nOptimal solution for y:")
# for f in 1:F
#     for i in 1:I
#         for j in 1:J
#             println("y[$f, $i, $j] = ", value(y[f, i, j]))
#         end
#     end
# end

# # for j in 1:J
# #     for i in 1:I
# #         for f in 1:F
# #             println("y[$f, $i, $j] = ", value(y[f, i, j]))
# #         end
# #     end
# # end

# println("\nOptimal solution for z:")
# for i in 1:I
#     println("z[$i] = ", value(z[i]))
# end

# println("\nOptimal solution for s:")
# for j in 1:J
#     println("s[$j] = ", value(s[j]))
# end
